# Data Science for Geoscience

Let's use some standard Machine Learning tools available in Python packages to analyse some data.

We have a dataset (from Butterworth et al 2016) with a collection of tectonomagmatic parameters associated with the time and location of porphyry copper deposits. We want to determine which of these (if any) parameters are geologically important (or at least statistically significant) in relation to the formation of porphyry coppers.

Run the next cell to see an animation representing this data


In [ ]:
from IPython.display import Image
Image(filename="../data/MullerConvergenceSmall.gif")

### Now, import most of the modules we need
By convention module loads go at the top of your workflows.

In [ ]:
import pandas #For dealing with data structures
import numpy as np #Data array manipulation
import scipy #Scientific Python, has lots of useful tools
import scipy.io #A specific sub-module for input/output of sci data

#scikit-learn tools to perform machine learning classification
from sklearn import cross_validation
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

#For making pretty figures
import matplotlib.pyplot as plt 

#For easy geographic projections on a map
from mpl_toolkits.basemap import Basemap

#Import a set of tools we have made ourselves
from Utils_coreg import *

### Now load in the data

In [ ]:
#Use pandas to load in the machine learning dataset
ml_data=pandas.read_csv("../data/ml_data_points.csv",index_col=0)

In [ ]:
#Print out the dataset so we can see what it looks like
ml_data

There are 21 columns (python (usually) counts from 0) representing different parameters. Some of these parameters may be useful for us. Some are not. The final column contains a binary flag representing whether there is a known porphyry copper deposit at that location or not. The "non-deposits" are required to train our Machine Learning classifier what a porphyry deposit looks like, and also, what a porphyry deposit doesn't look like!

### Now let's perform our machine learning binary classification.

In [ ]:
#Change data format to numpy array for easy manipulation
ml_data_np=ml_data.values

#Set the indices of the parameters (features) to include in the ML
#params=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
# Alternatively try any 4 features you'd like to include!
params=[6,9,14,17] 


#Save the number of parameters we have chosen
datalength=len(params)

#Normalise the data for Machine Learning
ml_data_norm=preprocessing.scale(ml_data_np[:,params])

#Create a 'feature vector' and a 'target classification vector'
features=ml_data_norm
targets=ml_data_np[:,20]

#Print out some info about our final dataset
print("Shape of ML data array: ", ml_data_norm.shape)
print("Positive (deposits) examples: ",np.shape(ml_data_np[ml_data_np[:,20]==1,:]))
print("Negative (non-deposits) examples: ",np.shape(ml_data_np[ml_data_np[:,20]==0,:]))

In [ ]:
print('Make the classifiers')

print('Random Forest...')
#create and train the random forest
#multi-core CPUs can use: rf = RandomForestClassifier(n_estimators=100, n_jobs=2)
#n_estimators use between 64-128 doi: 10.1007/978-3-642-31537-4_13
rf = RandomForestClassifier(n_estimators=128, n_jobs=1,class_weight=None)
rf.fit(features,targets)
print("Done RF")

scores = cross_validation.cross_val_score(rf, features,targets, cv=10)
print("RF Scores: ",scores)
print("SCORE Mean: %.2f" % np.mean(scores), "STD: %.2f" % np.std(scores), "\n")

print("Targets (expected result):")
print(targets)

print("Prediction (actual result):")
print(rf.predict(features))

In [ ]:
#Make a list of labels for our chosen features
paramColumns=np.array(ml_data.columns)
paramLabels=paramColumns[params].tolist()

#Create a new figure
fig, ax = plt.subplots()

#Plot the bar graph
rects=ax.bar(np.arange(0, datalength, step=1),rf.feature_importances_)

#Label the axes
ax.set_xticks(np.arange(0, datalength, step=1))
ax.set_xticklabels(paramLabels,rotation=90)
ax.set_ylabel('Feature Importance')

#Print the feature importance to compare with plot
np.set_printoptions(precision=3,suppress=True)
print("Importance \t Feature")
for i,label in enumerate(paramLabels):
    print("%1.3f \t\t %s" % (rf.feature_importances_[i],label))

plt.show()


Now if we can measure the tectonomagmatic properties at some point. Based on our trained classifer we can predict a probability that porphyry copper deposits have formed

In [ ]:
#Apply the trained ML to our gridded data to determine the probabilities at each of the points
print('RF...')
pRF=np.array(rf.predict_proba(features))
print("Done RF")

## Maps!

In [ ]:
filename="../data/EarthByte_Zahirovic_etal_2016_ESR_r888_AgeGrid-0.nc"
data = scipy.io.netcdf.netcdf_file(filename,'r')
data.variables

In [ ]:
varX=data.variables['lon'][:]
varY=data.variables['lat'][:]
varZ=np.array(data.variables['z'][:])
data.close()

In [ ]:
#Make a figure object
plt.figure()

#Get the axes of the current figure, for manipulation
ax = plt.gca()

#Create a colormap from a predefined function
age_cmap=colormap_age()

#Put down the main dataset
im=ax.imshow(varZ,vmin=0,vmax=200,extent=[0,360,-90,90],origin='lower',aspect=1,cmap=age_cmap)

#Make a colorbar
cbar=plt.colorbar(im,fraction=0.025,pad=0.05,ticks=[0, 150],extend='max')
cbar.set_label('Age (Ma)', rotation=0)

#Clean up the default axis ticks
plt.yticks([-90,-45,0,45,90])
plt.xticks([0,90,180,270,360])

#Put labels on the figure
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

#Put a title on it
plt.title("Global Seafloor Age Grid \n (Zahirovic et al. 2016)")

plt.show()

### For loops plotting shapefiles

In [ ]:
#Load in plate polygons for plotting
topologyFile='../data/topology_platepolygons_0.00Ma.shp'
[recs,shapes,fields,Nshp]=readTopologyPlatepolygonFile(topologyFile)
for i, nshp in enumerate(range(Nshp)):
    #if nshp!=35 and nshp!=36 and nshp!=23:
    #These are the plates that cross the dateline and cause 
        #banding errors
        polygonShape=shapes[nshp].points
        poly=np.array(polygonShape)
        plt.plot(poly[:,0], poly[:,1], c='k',zorder=1)
        
plt.show()

In [ ]:
filename="../data/topodata.nc"
data = scipy.io.netcdf.netcdf_file(filename,'r')

data.variables

In [ ]:
topoX=data.variables['X'][:]
topoY=data.variables['Y'][:]
topoZ=np.array(data.variables['elev'][:])
data.close()

### Make a prettier map

In [ ]:
#Make the base figure
fig = plt.figure(figsize=(8,6),dpi=300)
pmap=Basemap(projection='merc',llcrnrlat=-55,urcrnrlat=10,\
            llcrnrlon=-85,urcrnrlon=-30,lat_ts=0,resolution='c')

pmap.drawmapboundary(fill_color='white')
#pmap.fillcontinents(color='grey', lake_color='white', zorder=0)
pmap.drawmeridians(np.arange(0, 360, 20),labels=[0,0,0,1],fontsize=6,color='white')
pmap.drawparallels(np.arange(-90, 90, 10),labels=[1,0,0,0],fontsize=6,color='white')

#Plot a background topography map
lons=topoX
lats=topoY
lons, lats = np.meshgrid(lons,lats)
im1 = pmap.pcolormesh(lons,lats,topoZ,shading='flat',cmap=plt.cm.gist_earth,latlon=True)
cb = pmap.colorbar(im1,"right", size="5%", pad="2%",extend='both',ticks=[-2000,0,2000])
plt.clim(-5000,4000)
cb.set_label('Height (m)',labelpad=-25)

#Plot a background Agegrid map (on top of topo)
lons=varX
lats=varY
lons, lats = np.meshgrid(lons,lats)
#Make the NaNs clear
intensity = np.ma.masked_where(np.isnan(varZ), varZ)
im1 = pmap.pcolormesh(lons,lats,intensity,shading='flat',cmap=age_cmap,latlon=True)
cb = pmap.colorbar(im1,"right", size="5%", pad="60%",extend='max',ticks=[0,200])
plt.clim(0,200)
cb.set_label('Age (Ma)',labelpad=-20)


#Load in plate polygons for plotting
topologyFile='../data/topology_platepolygons_0.00Ma.shp'
[recs,shapes,fields,Nshp]=readTopologyPlatepolygonFile(topologyFile)
for i, nshp in enumerate(range(Nshp)):
    if nshp!=35 and nshp!=36 and nshp!=23:
    #These are the plates that cross the dateline and cause 
        #banding errors
        polygonShape=shapes[nshp].points
        poly=np.array(polygonShape)
        xh, yh = pmap(poly[:,0], poly[:,1])
        plt.plot(xh, yh, c='w',zorder=1)


#Plot the ore deposit Age
# xh, yh = pmap(ml_data_np[ml_data_np[:,-1]==1,0],ml_data_np[ml_data_np[:,-1]==1,1])
# l2 = pmap.scatter(xh, yh, 20, marker='.',c=ml_data_np[ml_data_np[:,-1]==1,4],cmap=plt.cm.hsv,zorder=3)
# cbar=pmap.colorbar(l2,location='bottom',pad="5%",ticks=[0,50,100,150],extend='max')
# plt.clim(0,170)
# cbar.set_label('Age of Deposit (Ma)')

#Plot the ore deposit probability
xh, yh = pmap(ml_data_np[ml_data_np[:,-1]==1,0],ml_data_np[ml_data_np[:,-1]==1,1])
l2 = pmap.scatter(xh, yh, 20, marker='.',c=pRF[:147,1],cmap=plt.cm.copper,zorder=3)
#l2 = pmap.scatter(xh, yh, 20, marker='.',edgecolor='dimgrey',linewidth=0.5,c=pRF[:147,1],cmap=plt.cm.copper,zorder=3)
cbar=pmap.colorbar(l2,location='bottom',pad="5%",ticks=[0,0.5,1])
plt.clim(0,1)
cbar.set_label('Prediction Probability (%)')

   
plt.show()

# Datasets